In [1]:
%reload_ext autoreload
%autoreload 2

from ripple_heterogeneity.utils import (
    functions,
    loading,
    batch_analysis,
    add_new_deep_sup,
    custom_plots,
    compress_repeated_epochs
)
from ripple_heterogeneity.readout import assembly_multi_region, assembly_multi_region_member_ratemap_corr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator
import os
import nelpy as nel
from statannot import add_stat_annotation
from statannotations.Annotator import Annotator
import glob
import pickle
import itertools
from ripple_heterogeneity.place_cells import maps
from scipy import stats

functions.set_plotting_defaults()
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
fig_save_path = r'Z:\home\ryanh\projects\ripple_heterogeneity\figures\panels'

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [16]:
save_path = r"Z:\home\ryanh\projects\ripple_heterogeneity\assembly_multi_region_25ms_dt_sleep_compress_v1"
sessions = glob.glob(save_path + os.sep + "*.pkl")
sessions_df = pd.DataFrame()
sessions_df["basepath"] = sessions

In [17]:
sessions_df

,basepath
0,Z:\home\ryanh\projects\ripple_heterogeneity\as...
1,Z:\home\ryanh\projects\ripple_heterogeneity\as...
2,Z:\home\ryanh\projects\ripple_heterogeneity\as...
3,Z:\home\ryanh\projects\ripple_heterogeneity\as...
4,Z:\home\ryanh\projects\ripple_heterogeneity\as...
...,...
107,Z:\home\ryanh\projects\ripple_heterogeneity\as...
108,Z:\home\ryanh\projects\ripple_heterogeneity\as...
109,Z:\home\ryanh\projects\ripple_heterogeneity\as...
110,Z:\home\ryanh\projects\ripple_heterogeneity\as...


In [14]:
results = assembly_multi_region_member_ratemap_corr.run(r"Z:\home\ryanh\projects\ripple_heterogeneity\assembly_multi_region_25ms_dt_sleep_compress_v1\Z__Data_ORproject_OR18_day2.pkl")


c:\Users\Cornell\anaconda3\envs\ripple_diversity\lib\site-packages\numpy\core\fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
c:\Users\Cornell\anaconda3\envs\ripple_diversity\lib\site-packages\numpy\core\_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
c:\Users\Cornell\anaconda3\envs\ripple_diversity\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
results.query("brainRegion_ref=='CA1' & deepSuperficial_ref == 'Deep' & brainRegion_tar=='MEC'")

sns.boxplot(data=results.query("is_member_sig_ref & is_member_sig_tar"),x="deepSuperficial_ref",y="spatial_corr")

In [21]:
save_path = r"Z:\home\ryanh\projects\ripple_heterogeneity\assembly_multi_region_member_ratemap_corr_v1"

batch_analysis.run(
    sessions_df,
    save_path,
    assembly_multi_region_member_ratemap_corr.run,
    parallel=False,
    skip_if_error=False,
    verbose=True,
)

ccgs, label_df = assembly_multi_region_member_ratemap_corr.load_results(save_path)

  0%|          | 0/112 [00:00<?, ?it/s]

Z:\home\ryanh\projects\ripple_heterogeneity\assembly_multi_region_25ms_dt_sleep_compress_v1\Z__Data_AYAold_AYA7_day19.pkl
Z:\home\ryanh\projects\ripple_heterogeneity\assembly_multi_region_25ms_dt_sleep_compress_v1\Z__Data_AYAold_AYA7_day20.pkl
Z:\home\ryanh\projects\ripple_heterogeneity\assembly_multi_region_25ms_dt_sleep_compress_v1\Z__Data_AYAold_AYA7_day24.pkl


c:\Users\Cornell\anaconda3\envs\ripple_diversity\lib\site-packages\numpy\core\fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
c:\Users\Cornell\anaconda3\envs\ripple_diversity\lib\site-packages\numpy\core\_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
c:\Users\Cornell\anaconda3\envs\ripple_diversity\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
d:\github\ripple_heterogeneity\ripple_heterogeneity\readout\assembly_multi_region_member_ratemap_corr.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  2%|▏         | 2/112 [06:23<5:51:28, 191.71s/it]


IndexError: index 0 is out of bounds for axis 0 with size 0